In [5]:
include("ga_exp.jl")

Imports completed
All models read in successfully!


Initial conditions determined, starting simulation...

┌ Info: fmin: 100 / 100 trials carried out
└ @ TreeParzen.API C:\Users\Charlotte\.julia\packages\TreeParzen\gdoI0\src\API.jl:176
┌ Info: Successfully completed fmin 
└ @ TreeParzen.API C:\Users\Charlotte\.julia\packages\TreeParzen\gdoI0\src\API.jl:231



Simulation complete!

"ode_data.csv"

In [6]:
#Run single simulation
A = [[0, 0, 1], [0, 0, 1]] #open loop
W = [[2., 3.328086, 0.000041], [2., 5.070964, 0.000227]] #Optimal from MSc work - glucaric_acid_singlearch.csv
params = [A, W]
bo_data, fba_data, ode_data = single_run(params, 100, 86400)

CSV.write("fba_data.csv", fba_data)
CSV.write("ode_data.csv", ode_data)

In [1]:
plot(fba_data.lam)

UndefVarError: UndefVarError: fba_data not defined